Sentence similarity
Dataset: STS

In [ ]:
import torch
from datasets import load_dataset, Dataset
import nltk
import re
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader
from sentence_transformers import SentenceTransformer, SentenceTransformerTrainer, losses

from nltk.corpus import words
from nltk.corpus import stopwords
nltk.download('stopwords', quiet=True)
nltk.download('words', quiet=True)
nltk.download('wordnet', quiet=True)
# vocab = set(words.words())
from nltk.stem import *

from preprocessing import clean_sentence, vocab, clean_dataset

In [ ]:
ds = load_dataset("tabilab/biosses")

In [ ]:
train_test = ds['train'].train_test_split(test_size=0.2, seed=42)
train_validate = train_test['train'].train_test_split(test_size=0.2, seed=42)
train_set = train_validate['train']
validate_set = train_validate['test']
test_set = train_test['test']

In [ ]:
stemmer = PorterStemmer()

In [ ]:
stemmed_words = set(map(lambda word: stemmer.stem(word), words.words()))

In [ ]:
cleaned_train, unknown_words = clean_dataset(train_set, stemmed_words)
cleaned_validation, _ = clean_dataset(validate_set, stemmed_words)
cleaned_test, _ = clean_dataset(test_set, stemmed_words)

In [ ]:
print(unknown_words)
print(len(unknown_words))

{'pRb', 'Arp4-related', 'miR-16−1', 'Transfection', 'RIP3', 'et', 'non-neural', '3', 'PPP', 'OSCC', 'downregulated', '22]', '(Mirus)', 'AML', 'ESCs', 'up-regulation', '95%', 'receptor-interacting', 'ligase', 'upregulation', 'upregulated', 'protein-α', 'rate-limiting', 'miR-24', 'NH2-terminal', 'hr', 'Kras-driven', 'let-7)', 'miR-223', 'IL-1', '1', 'CCAAT/enhancer', 'MiR-155', 'S2)', 'TFs', 'Sox11', 'SOX2', 'review)', 'Aurora-A', '[18]', 'IDH1/IDH2', 'signal-regulated', 'TET', 'proteasome', 'RNAi-mediated', 'I/A', 'miR-34a', '(GAPs)', 'GTPase', 'miR-143', 'hWts2', 'MiR-223', '(BRG-associated', 'microtubule-nucleating', '(GEFs)', '“cyclin', 'interest;', '(RIP', 'IDH1', 'RNAi', 'miR-204', 'TEL-AML1-positive', 'Arp', 'ductal', 'Up-regulation', 'OCT4', 'dose-dependent', 'GATA6', 'HeLa', '<20%', 'ubiquitin', '(together', 'TargetScan', 'RXXL', 'oncogene', 'Co-transfection', 'EC', 'Neuro-2a', 'qRT-PCR', 'RIP1', 'miR-146b', 'large-scale', '7.4-fold', 'EGFR', '43', 'K-Ras-driven', 'BCL-XL/MEK', 

In [ ]:
torch.cuda.empty_cache()

In [ ]:
from sentence_transformers import SentenceTransformer, SentenceTransformerTrainingArguments
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator, SimilarityFunction
from sentence_transformers.losses import CoSENTLoss

# Load a model to train/finetune
model = SentenceTransformer('all-MiniLM-L6-v2', device="cuda")

# Initialize the CoSENTLoss
# This loss requires pairs of text and a float similarity score as a label
loss = CoSENTLoss(model)

In [ ]:
train_s1 = cleaned_train['sentence1']
train_s2 = cleaned_train['sentence2']
validation_s1 = cleaned_validation['sentence1']
validation_s2 = cleaned_validation['sentence2']

In [ ]:
# train_emb1 = model.encode(train_s1, normalize_embeddings=True)
# train_emb2 = model.encode(train_s2, normalize_embeddings=True)
# validation_emb1 = model.encode(validation_s1, normalize_embeddings=True)
# validation_emb2 = model.encode(validation_s2, normalize_embeddings=True)

In [ ]:
# validation_similarities = model.similarity_pairwise(validation_emb1, validation_emb2)
# print(validation_similarities)
ground_truth_validation = cleaned_validation["score"]
# print(ground_truth_validation)
# print(loss.forward(
#     {"sentence1": cleaned_validation["sentence1"],
#      "sentence2": cleaned_validation["sentence2"]}, 
#     ground_truth_validation))

In [ ]:
test_similarities = model.similarity_pairwise(
    model.encode(cleaned_test['sentence1'], normalize_embeddings=True),
      model.encode(cleaned_test['sentence2'], normalize_embeddings=True))
print(test_similarities)
ground_truth_test = cleaned_test["score"]
print(ground_truth_test)

tensor([0.6227, 0.7996, 0.4542, 0.7373, 0.5717, 0.9421, 0.4272, 0.3671, 0.2034,
        0.7460, 0.5736, 0.5206, 0.8329, 0.6441, 0.5521, 0.7422, 0.3377, 0.7593,
        0.4291, 0.3967])
[0.20000004768371582, 0.7999999523162842, -0.30000001192092896, 0.6000000238418579, 0.5, 1.0, 0.0, -0.30000001192092896, -0.10000002384185791, 0.7000000476837158, 0.0, 0.0, 0.8999999761581421, 0.7000000476837158, 0.7999999523162842, 0.5, 0.20000004768371582, 0.6000000238418579, 0.20000004768371582, -0.30000001192092896]


C:\Users\pc\AppData\Roaming\Python\Python312\site-packages\transformers\models\bert\modeling_bert.py:439: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


In [ ]:
args = SentenceTransformerTrainingArguments(
    # Required parameter:
    output_dir="models/all-MiniLM-L6-v2",
    # Optional training parameters:
    num_train_epochs=300,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=100,
    learning_rate=1e-6,
    warmup_ratio=0.1,
    fp16=True,  # Set to False if you get an error that your GPU can't run on FP16
    bf16=False,  # Set to True if you have a GPU that supports BF16
    # batch_sampler=BatchSamplers.NO_DUPLICATES,  # losses that use "in-batch negatives" benefit from no duplicates
    # Optional tracking/debugging parameters:
    eval_strategy="steps",
    eval_steps=0.05,
    save_strategy="steps",
    save_steps=0.05,
    save_total_limit=3,
    load_best_model_at_end=True,
    logging_steps=0.05,
    run_name='all-MiniLM-L6-v2',  # Will be used in W&B if `wandb` is installed
)

In [ ]:
dev_evaluator = EmbeddingSimilarityEvaluator(
    sentences1=cleaned_validation["sentence1"],
    sentences2=cleaned_validation["sentence2"],
    scores=cleaned_validation["score"],
    name="BIOSSES_validate",
)
test_evaluator = EmbeddingSimilarityEvaluator(
    sentences1=cleaned_test["sentence1"],
    sentences2=cleaned_test["sentence2"],
    scores=cleaned_test["score"],
    name="BIOSSES_test",
)
results = dev_evaluator(model)

In [ ]:
results

{'BIOSSES_validate_pearson_cosine': 0.8034244246101594,
 'BIOSSES_validate_spearman_cosine': 0.7961717622928737}

In [ ]:
trainer = SentenceTransformerTrainer(
    model=model,
    train_dataset=cleaned_train,
    eval_dataset=cleaned_validation,
    loss=loss,
    args=args,
    evaluator=dev_evaluator,
)
trainer.train()

  0%|          | 0/1600 [00:00<?, ?it/s]

{'loss': 1.7895, 'grad_norm': 132.9878387451172, 'learning_rate': 5e-07, 'epoch': 5.0}


  0%|          | 0/1 [00:00<?, ?it/s]

{'eval_loss': 7.377745628356934, 'eval_BIOSSES_validate_pearson_cosine': 0.8051602243791965, 'eval_BIOSSES_validate_spearman_cosine': 0.784310730452384, 'eval_runtime': 0.1367, 'eval_samples_per_second': 117.028, 'eval_steps_per_second': 7.314, 'epoch': 5.0}


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

{'eval_loss': 5.833976745605469, 'eval_BIOSSES_validate_pearson_cosine': 0.8515467225051898, 'eval_BIOSSES_validate_spearman_cosine': 0.8465811476149551, 'eval_runtime': 0.1328, 'eval_samples_per_second': 120.503, 'eval_steps_per_second': 7.531, 'epoch': 95.0}
{'loss': 0.4925, 'grad_norm': 11.6255464553833, 'learning_rate': 0.0, 'epoch': 100.0}


  0%|          | 0/1 [00:00<?, ?it/s]

{'eval_loss': 5.8351335525512695, 'eval_BIOSSES_validate_pearson_cosine': 0.851519633019667, 'eval_BIOSSES_validate_spearman_cosine': 0.8465811476149551, 'eval_runtime': 0.2451, 'eval_samples_per_second': 65.28, 'eval_steps_per_second': 4.08, 'epoch': 100.0}
{'train_runtime': 174.7099, 'train_samples_per_second': 36.632, 'train_steps_per_second': 9.158, 'train_loss': 0.8619360375404358, 'epoch': 100.0}


TrainOutput(global_step=1600, training_loss=0.8619360375404358, metrics={'train_runtime': 174.7099, 'train_samples_per_second': 36.632, 'train_steps_per_second': 9.158, 'total_flos': 0.0, 'train_loss': 0.8619360375404358, 'epoch': 100.0})

In [ ]:
test_results = test_evaluator(model)
test_results

{'BIOSSES_test_pearson_cosine': 0.8518288612627938,
 'BIOSSES_test_spearman_cosine': 0.8237669743411928}

In [ ]:
model.save_pretrained("models/all-MiniLM-L6-v2/final")

In [ ]:
cleaned_train

Dataset({
    features: ['sentence1', 'sentence2', 'score'],
    num_rows: 64
})

In [ ]:
def evaluate_model(model_, test_set_, name="BIOSSES_test"):
    test_evaluator = EmbeddingSimilarityEvaluator(
        sentences1=test_set_["sentence1"],
        sentences2=test_set_["sentence2"],
        scores=test_set_["score"],
        name=name,
    )
    return test_evaluator(model)

In [ ]:
print(evaluate_model(model, cleaned_test)['BIOSSES_test_pearson_cosine'])
print(evaluate_model(model, cleaned_train)['BIOSSES_test_pearson_cosine'])
print(evaluate_model(model, cleaned_validation)['BIOSSES_test_pearson_cosine'])

0.8518288612627938
0.9721099006701728
0.8515467225051898
